In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVR
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.preprocessing import FunctionTransformer
from copy import copy
from sklearn.model_selection import TimeSeriesSplit

tpot_data = pd.read_csv('Combined.csv')
features = tpot_data.drop(['Forecast','Q-PS_MGD_prev_24h_2'], axis = 1)
features = features.drop(['Timestamp'], axis=1)
feature_list = list(features.columns)
features = np.array(features)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['Forecast'], shuffle=False, test_size=0.2)
cv=TimeSeriesSplit(gap=300)

# Average CV score on the training set was: -334.8915498690726
exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        FunctionTransformer(copy)
    ),
    SelectPercentile(score_func=f_regression, percentile=61),
    LinearSVR(C=0.001, dual=False, epsilon=0.1, loss="squared_epsilon_insensitive", tol=1e-05)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 1)

exported_pipeline.fit(training_features, training_target)

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('functiontransformer-1',
                                                 FunctionTransformer(func=<function copy at 0x7fa7fcd59760>)),
                                                ('functiontransformer-2',
                                                 FunctionTransformer(func=<function copy at 0x7fa7fcd59760>))])),
                ('selectpercentile',
                 SelectPercentile(percentile=61,
                                  score_func=<function f_regression at 0x7fa68bb139c0>)),
                ('linearsvr',
                 LinearSVR(C=0.001, dual=False, epsilon=0.1,
                           loss='squared_epsilon_insensitive', random_state=1,
                           tol=1e-05))])

In [2]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import math

# Predict using the fitted model
y_train_ml = exported_pipeline.predict(training_features)
y_test_ml = exported_pipeline.predict(testing_features)

# Calculate training and testing error
mse_train_ml = mean_squared_error(training_target, y_train_ml)
mape_train_ml = mean_absolute_percentage_error(training_target, y_train_ml)
r2_train_ml = r2_score(training_target, y_train_ml)
rmse_train = math.sqrt(mse_train_ml)
mse_test_ml = mean_squared_error(testing_target, y_test_ml)
mape_test_ml = mean_absolute_percentage_error(testing_target, y_test_ml)
r2_test_ml = r2_score(testing_target, y_test_ml)
rmse_test = math.sqrt(mse_test_ml)

# Print training and testing error
print(f"Training Mean Squared Error (MSE): {round(mse_train_ml,3)}")
print(f"Training Root Mean Squared Error (MSE): {round(rmse_train,3)}")
print(f"Training Mean Absolute Percentage Error (MAPE): {round(mape_train_ml,3)}")
print(f"Training R-squared (R2) Score: {round(r2_train_ml,3)}")
print(f"Testing Mean Squared Error (MSE): {round(mse_test_ml,3)}")
print(f"Testing Root Mean Squared Error (MSE): {round(rmse_test,3)}")
print(f"Testing Mean Absolute Percentage Error (MAPE): {round(mape_test_ml,3)}")
print(f"Testing R-squared (R2) Score: {round(r2_test_ml,3)}")

def adjusted_r2(r2, n, k):
    return 1 - ((1 - r2) * (n - 1)) / (n - k - 1)

n = len(testing_features)  # Number of samples in test data
k = len(feature_list)  # Number of predictors in the model

# Calculate Adjusted R-squared
adjusted_r2_value = adjusted_r2(r2_test_ml, n, k)
print("Adjusted R-squared:", round(adjusted_r2_value,3))
print("Number of variables:",k)
TeEI = (mape_test_ml*mape_test_ml*rmse_test)/adjusted_r2_value
print("TeEI:", round(TeEI,3))

Training Mean Squared Error (MSE): 365.107
Training Root Mean Squared Error (MSE): 19.108
Training Mean Absolute Percentage Error (MAPE): 0.156
Training R-squared (R2) Score: 0.724
Testing Mean Squared Error (MSE): 531.543
Testing Root Mean Squared Error (MSE): 23.055
Testing Mean Absolute Percentage Error (MAPE): 0.157
Testing R-squared (R2) Score: 0.711
Adjusted R-squared: 0.711
Number of variables: 38
TeEI: 0.798
